# Projecto Integrador

Primer acercamiento a trabajar con el concepto de web Scrapping. El proyecto fué propuesto en clase por el maestro Gonzalo del Rio, durante el primer módulo del Bootcamp de Henry.

El proyecto consiste en hacer WebScrapping al sitio web https://cuspide.com/ para extraer la información de la sección de "100 libros más vendidos de la semana" en un formato permita hacer el procesamiento con Pandas y crear finalmente un archivo csv de salida.

***

### Para empezar:

#### 1) Preparar el ambiente. 

Se importan las librerías de procesamiento de los datos con las que se va a trabajar.

- Se usa Pandas para el procesamiento y visualización de los datos. **Se requiere crear un DataFrame.**
- Se usa el módulo `requests` para enviar requerimentos HTTP al sitio que nos interesa. Como respuesta queremos obtener el HTML crudo de la página.
- Se usa `Beautiful Soup` para leer el HTML en una estructura que sea útil para extraer la información.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [3]:
# Se guarda la url en una variable con el mismo nombre
url = 'https://cuspide.com/100-mas-vendidos/' 

# se hace el request usando el metodo .get() y nos devuelta un objeto respuesta --> requests.Resopnse (De requests, para ampliar veáse la documentación de la librería).
response = requests.get(url) 

# Las respuestas tienen el atributo .content, en este caso con el html crudo del sitio, se guarda en una variable
html_content = response.content

### 2) BeautifulSoup 

Beautiful soup interpreta el contenido html crudo y lo organiza en estructura jerárquica, parecido a como lo hace el navegador. Con la estructura de etiquetas donde podemos analizar esa jerarquía ya se puede empezar a estraer la información. 

Esa es la magia de Beautiful soup :D

In [5]:
# Definimos la "sopa de etiquetas" donde se encuentra todo el contenido html
soup = BeautifulSoup(html_content, 'lxml')

Luego de inspeccionar la página, se observó que los titulos están contenidos en una etiqueta `<h3>` con el atributo de clase "product-title".

**Nota**: El método `find_all()` retorna un objeto tipo ResultSet que contiene todas las etiquetas que cumplan con los parámetros que se pasen por la función. Este ResultSet se puede iterar para trabajar con cada etiqueta específica.

In [ ]:
# Buscamos las etiquetas donde se encuentran los titulos.
title_tags = soup.find_all("h3", class_='product-title')

for tag in title_tags: # Iteramos sobre el conjunto que obtuvimos para imprimir
   print(tag)

Teniendo las etiquetas, podemos guardarlas en una lista (usando list comprehension).

In [9]:
# List comp para iterar sobre las etiquetas de los títulos y obtener el string del título para cada uno.
titulos = [tag.string for tag in title_tags]
titulos

['LA CASA NEVILLE . LA FORMIDABLE SEÑORITA  MANON',
 'EL PROBLEMA FINAL',
 'TESIS SOBRE UNA DOMESTICACION',
 'ARTIFICIAL',
 'HOLLY',
 'BEYOND THE STORY ( EDICION EN ESPA/OL )',
 'EL VUELO DE LA LIBELULA',
 'EL VIENTO CONOCE MI NOMBRE',
 'UNA FAMILIA ANORMAL',
 'DESTROZA ESTE DIARIO ( A TODO COLOR )',
 '73 MARGARITAS',
 'EL PODER DE LAS PALABRAS',
 'HOROSCOPO CHINO 2024',
 'EL PODER DEL AHORA',
 'UN VECINO ANORMAL  : Y EL LADRON DEL CHOCOLATE',
 'ANTES DE QUE SE ENFRIE EL CAFE',
 'LA CRISIS DE LA NARRACION',
 'DEJA DE SER TU',
 'NOSOTROS DOS EN LA TORMENTA',
 'LA ARMADURA DE LA LUZ',
 'EL HOMBRE EN BUSCA DE SENTIDO',
 'LOS CUATRO ACUERDOS',
 'BORN Y QUIETO',
 'MIS DIAS EN LA LIBRERIA MORISAKI',
 'MAFALDA : TODAS LAS TIRAS',
 'PORQUE DEMASIADO NO ES SUFICIENTE',
 'LA PACIENTE SILENCIOSA',
 'ADIOS CACHORRA',
 'LAS NI/AS DEL NARANJEL',
 'ELON MUSK',
 '1966 DE ILLIA A ONGANIA',
 'EL AMOR ES IMPOSIBLE',
 'LA FELICIDAD CABE EN UNA TAZA DE CAFE',
 'MUCHAS VIDAS , MUCHOS MAESTROS',
 'LAS COSAS 


... Por inspección tambíen puede verse que los url de cada libro están contenidos en una etiqueta "hija" dentro de las mismas etiquetas de los títulos.

**Nota**: El objeto etiqueta nos permite acceder a las etiquetas que tiene por dentro (estructura jerárquica de arbol). Con el atritubo .attrs podemos ver un diccionario con los atributos de la etiqueta.

In [10]:
# List comp para iterar sobre las etiquetas de los títulos y obtener el valor del atritubo "href" que contiene los urls
urls = [tag.a.attrs['href'] for tag in title_tags]
urls

['https://cuspide.com/producto/la-casa-neville-la-formidable-senorita-manon/',
 'https://cuspide.com/producto/el-problema-final/',
 'https://cuspide.com/producto/tesis-sobre-una-domesticacion/',
 'https://cuspide.com/producto/artificial-2/',
 'https://cuspide.com/producto/holly/',
 'https://cuspide.com/producto/beyond-the-story-edicion-en-espa-ol/',
 'https://cuspide.com/producto/el-vuelo-de-la-libelula/',
 'https://cuspide.com/producto/el-viento-conoce-mi-nombre/',
 'https://cuspide.com/producto/una-familia-anormal-5/',
 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/',
 'https://cuspide.com/producto/73-margaritas/',
 'https://cuspide.com/producto/el-poder-de-las-palabras/',
 'https://cuspide.com/producto/horoscopo-chino-2024/',
 'https://cuspide.com/producto/el-poder-del-ahora/',
 'https://cuspide.com/producto/un-vecino-anormal-y-el-ladron-del-chocolate/',
 'https://cuspide.com/producto/antes-de-que-se-enfrie-el-cafe/',
 'https://cuspide.com/producto/la-crisis-de-la-

#### Para los precios se usó un método de distinto (para ilustrar que hay diferentes formas de hacerlo).

En lugar de find_all, puede usarse el método selector de css que de igual manera es muy cómodo para seleccionar las etiquetas por clases, estilos y id.

In [12]:
# Seleccionar las etiquetas de los precios
price_tags = soup.css.select('.product-price')

for tag in price_tags:
    tag.select(".woocommerce-Price-currencySymbol")[0].decompose()

In [16]:
price_column = [tag.string[1:] for tag in price_tags]
price_column

['11.900,00',
 '10.699,00',
 '9.900,00',
 '7.999,00',
 '15.999,00',
 '9.999,00',
 '8.799,00',
 '10.699,00',
 '4.199,00',
 '11.200,00',
 '6.999,00',
 '10.599,00',
 '7.999,00',
 '7.199,00',
 '5.199,00',
 '9.699,00',
 '6.800,00',
 '13.390,00',
 '11.599,00',
 '19.999,00',
 '6.890,00',
 '8.490,00',
 '11.800,00',
 '5.850,00',
 '14.100,00',
 '8.900,00',
 '9.999,00',
 '9.000,00',
 '9.999,00',
 '14.999,00',
 '9.999,00',
 '11.200,00',
 '5.999,00',
 '7.199,00',
 '10.950,00',
 '8.500,00',
 '4.290,00',
 '5.990,00',
 '5.399,00',
 '6.999,00',
 '3.749,00',
 '6.490,00',
 '5.490,00',
 '5.700,00',
 '13.900,00',
 '15.300,00',
 '3.299,00',
 '6.590,00',
 '6.490,00',
 '10.500,00',
 '8.570,00',
 '7.250,00',
 '10.500,00',
 '10.500,00',
 '12.900,00',
 '12.500,00',
 '9.799,00',
 '9.200,00',
 '8.900,00',
 '5.950,00',
 '13.499,00',
 '7.500,00',
 '5.999,00',
 '7.199,00',
 '10.900,00',
 '5.940,00',
 '3.999,00',
 '10.950,00',
 '7.300,00',
 '6.350,00',
 '11.900,00',
 '6.599,00',
 '4.999,00',
 '8.600,00',
 '7.900,00',


Debido a que el precio en dólares se encuentra en cada url específico para cada libro, he creado funciones scrapper para esta tarea.
Las funciones están definidas dentro del módulo `functions.py`. Allí puede verse el script que se ha creado.

In [17]:
import functions

### obtener precio en usd de un libro para cada url específo del libro
usd_prices = [functions.get_us_price(url) for url in urls]

# Obtener las fechas, que tambien esán en cada url específico de cada libro.
dates = [functions.get_pub_date(url) for url in urls]

### Creación del DataFrame:

La siguiente consigna se trata de crear un dataFrame y posteriormente el csv de salida

In [21]:
df = pd.DataFrame(
    {
        'titulo': [str(title).title() for title in titulos],
        'url': [str(item) for item in urls],
        'precio_pesos': pd.Series(price_column),
        'precio_usd': pd.Series(usd_prices),
        'fecha_publicación': pd.to_datetime(dates, yearfirst=True) 
        
    }
)

/tmp/ipykernel_17815/1735649143.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  'fecha_publicación': pd.to_datetime(dates, yearfirst=True)


In [22]:
df.head(10)

,titulo,url,precio_pesos,precio_usd,fecha_publicación
0,La Casa Neville . La Formidable Señorita Manon,https://cuspide.com/producto/la-casa-neville-l...,"11.900,00","32,56",2023-09-29
1,El Problema Final,https://cuspide.com/producto/el-problema-final/,"10.699,00","29,27",2023-09-28
2,Tesis Sobre Una Domesticacion,https://cuspide.com/producto/tesis-sobre-una-d...,"9.900,00","27,09",2023-08-31
3,Artificial,https://cuspide.com/producto/artificial-2/,"7.999,00","21,89",2023-09-28
4,Holly,https://cuspide.com/producto/holly/,"15.999,00","43,77",2023-09-28
5,Beyond The Story ( Edicion En Espa/Ol ),https://cuspide.com/producto/beyond-the-story-...,"9.999,00","27,36",2023-08-29
6,El Vuelo De La Libelula,https://cuspide.com/producto/el-vuelo-de-la-li...,"8.799,00","24,07",2023-08-29
7,El Viento Conoce Mi Nombre,https://cuspide.com/producto/el-viento-conoce-...,"10.699,00","29,27",2023-06-06
8,Una Familia Anormal,https://cuspide.com/producto/una-familia-anorm...,"4.199,00","11,49",2023-07-28
9,Destroza Este Diario ( A Todo Color ),https://cuspide.com/producto/destroza-este-dia...,"11.200,00","30,64",2018-03-02


In [52]:
df['precio_pesos'] = df['precio_pesos'].apply(lambda item: item.replace(".","").replace(",",".")).apply(lambda item: float(item))

In [53]:
df['precio_usd'] = df['precio_usd'].apply(lambda item: item.replace(".","").replace(",",".")).apply(lambda item: float(item))

In [54]:
df

,titulo,url,precio_pesos,precio_usd,fecha_publicación
0,La Casa Neville . La Formidable Señorita Manon,https://cuspide.com/producto/la-casa-neville-l...,11900.0,32.56,2023-09-29
1,El Problema Final,https://cuspide.com/producto/el-problema-final/,10699.0,29.27,2023-09-28
2,Tesis Sobre Una Domesticacion,https://cuspide.com/producto/tesis-sobre-una-d...,9900.0,27.09,2023-08-31
3,Artificial,https://cuspide.com/producto/artificial-2/,7999.0,21.89,2023-09-28
4,Holly,https://cuspide.com/producto/holly/,15999.0,43.77,2023-09-28
...,...,...,...,...,...
86,Estas Para Mas,https://cuspide.com/producto/estas-para-mas/,9940.0,27.20,2022-03-10
87,Como Ganar Amigos E Influir Sobre Las Personas,https://cuspide.com/producto/como-ganar-amigos...,7199.0,19.70,2006-02-28
88,De Donde Vienen,https://cuspide.com/producto/de-donde-vienen/,5699.0,15.59,2023-03-29
89,Campeones Del Mundo,https://cuspide.com/producto/campeones-del-mundo/,6000.0,16.42,2023-08-16


In [55]:
df.to_csv('./libros_semana.csv', index=False)